In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

import pandas as pd
import re
from etl import enron_parse
import json

download email.csv from here: https://www.kaggle.com/wcukierski/enron-email-dataset

make sure to put it in the same directory as where NLU-email-title folder is at.

## Original Dataset

Total 517401 entries.

In [25]:
df = pd.read_csv('../data/raw/emails.csv')

In [3]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [38]:
len(df.message)

517401

## Parsed Dataset (Text)
- data originally 517401 entires
- reduce to only original emails (now total 293291 entries)
- cleaned the body to only have raw text left
- droped nan titles (276253)
- droped duplicated values (128706)
- droped those with 're:' or 'fw:' in the title (111044)

In [5]:
# parsed_All = enron_parse.parse_all(df.message)

100%|██████████| 517401/517401 [04:47<00:00, 1800.46it/s]


In [9]:
# df = pd.DataFrame(parsed_All).loc[:,['Subject','Body']]

In [10]:
# df.to_csv('ALL_data.csv',index=False)

In [215]:
df = enron_parse.filter_data('../data/interim/data_first_only_ALL.csv')

In [216]:
df.shape

(111044, 2)

In [217]:
df = df.sort_values(by='Subject',ascending=False)

There sre some level of reptitions throughout the body text.

In [218]:
df.iloc[-6]['Subject'], df.iloc[-6]['Body']

('"ANNOUNCING" A market that is Virtually UNTAPPED!!',
 'Would YOU  like to learn more about a company that has  - A product that is virtually UNTAPPED!  - Is Still widely UNKNOWN!  - A product EVERYONE needs!  - A product OVER  80% of Europeans have!  - A product UNDER 2% of the US has!  - Cost less than a cup of coffee a day!  - YOUR comissions are PAID DAILY!!!  - And is growing faster than MICROSOFT.... yes MICROSOFT!!!  Then send a email and see for YOURSELF!  Information residual28@yahoo.com  ________________________________________________ DISCLAIMER: ======================================== NOTE: This e-mail is not Spam. You are receiving this e-mail because you have previously requested more info on a Business Opportunity or we have shared same opt-in or safe e-mail list or we have exchanged business opportunities in the past. If you are no longer interested in exchanging business opportunities, please click on the link below  and put REMOVE ME in the subject line..  To be rem

In [219]:
df.iloc[-7]['Subject'], df.iloc[-7]['Body']

('"ANNOUNCING" a market that is Virtually UNTAPPED!!',
 'Would YOU  like to learn more about a company that has  - A product that is virtually UNTAPPED!  - Is Still widely UNKNOWN!  - A product EVERYONE needs!  - A product OVER  80% of Europeans have!  - A product UNDER 2% of the US has!  - Cost less than a cup of coffee a day!  - YOUR comissions are PAID DAILY!!!  - And is growing faster than MICROSOFT.... yes MICROSOFT!!!  Then send a email for more information and see for YOURSELF!  residual26@yahoo.com  ________________________________________________ DISCLAIMER: ======================================== NOTE: This e-mail is not Spam. You are receiving this e-mail because you have previously requested more info on a Business Opportunity or we have shared same opt-in or safe e-mail list or we have exchanged business opportunities in the past. If you are no longer interested in exchanging business opportunities, please send to the email below residual11@yahoo.com')

In [175]:
# df.to_csv('../data/interim/data_111044.csv', index=False)

In [324]:
'%19s'%'hello'

'              hello'

In [323]:
len('Old-Time / Historic')

19

In [198]:
# with open('../data/interim/data_111044.json', 'w') as f:
#     f.write(df.to_json(orient='records'))

In [3]:
data_111044 = pd.read_csv('../data/interim/data_111044.csv')

In [4]:
data_111044.head()

,Subject,Body
0,~~~~~~~TOP-NOTCH Attorneys For PENNIES A Day ~...,You Can Receive Thousands of Dollars in Legal ...
1,zev-program Second Notice of Availability of M...,The Second Notice of Availability of Modified ...
2,zev-program Second Notice,"Greetings, The Second Notice of Public Availa..."
3,zev-program California Fuel Cell Partnership R...,You are invited to attend or monitor live via ...
4,zev-program 15-day regulatory changes,"Greetings, The Notice of Public Availability ..."


sample a smaller set of data for model development

In [163]:
# small_df = df.sample(1000)

In [176]:
# small_df.to_csv('../data/interim/data_111044_1000.csv', index=False)

In [ ]:
# with open('../data/interim/data_111044_1000.json', 'w') as f:
#     f.write(small_df.to_json(orient='records'))

In [225]:
# saving as txt file?
# data_small = pd.read_csv('../data/interim/data_111044_1000.txt', sep='\t', names=['Body','Subject'])

read train, val, test split
- return body, subject pairs

In [276]:
DATA_PATH = '../data/interim/111044_1000/'
LARGE_DATA_PATH = '../data/interim/111044/'

In [272]:
train, val, test = enron_parse.read_data(DATA_PATH)

In [273]:
train.shape

(700, 2)

In [274]:
val.shape

(200, 2)

In [275]:
test.shape

(100, 2)

## Tokenized data


In [287]:
from torchtext import data, datasets, vocab
import spacy

In [294]:
def initialize_glove_vector(max_vectors):
    raw_vocab = vocab.GloVe(max_vectors=max_vectors)
    raw_vocab.itos = ['<unk>', '<pad>', '<s>', '</s>'] + raw_vocab.itos
    raw_vocab.stoi = defaultdict(vocab._default_unk_index)
    raw_vocab.stoi.update({tok: i for i, tok in enumerate(raw_vocab.itos)})
    emb_size = raw_vocab.vectors.size()[1]
    zero = torch.zeros(2, emb_size)
    vectors = torch.cat([zero, raw_vocab.vectors])
    raw_vocab.vectors = vectors
    return raw_vocab

glove_vocab_modify = initialize_glove_vector(max_vectors=100000)
# train.fields['x'].vocab = glove_vocab_modify
# vocabulary = train.fields['x'].vocab

In [72]:
def initialize_senna_vector():
    with open('senna_embed/words.lst' , 'r') as f:
        itos = [line.rstrip('\n') for line in f]
    stoi = defaultdict()
    counter = Counter(itos)
    raw_vocab = vocab.Vocab(counter)
    raw_vocab.itos = ['<unk>', '<pad>'] + itos
    raw_vocab.stoi = defaultdict(vocab._default_unk_index)
    raw_vocab.stoi.update({tok: i for i, tok in enumerate(raw_vocab.itos)})

In [147]:
class tokenizer:
    def __init__(self):
        self.stoi = {'<unk>':0, '<link>':1, '<pad>':2}
    def tokenize(self, x):
        return [i.split() for i in x]
    def build_vocab(self, x_tokenized):
        counter = Counter(np.concatenate(x_tokenized).flat)
        raw_vocab = torchtext.vocab.Vocab(counter)
        raw_vocab.itos = ['<unk>','<link>'] + raw_vocab.itos
        raw_vocab.stoi = defaultdict(torchtext.vocab._default_unk_index)
        raw_vocab.stoi.update({tok: i for i, tok in enumerate(raw_vocab.itos)})
        return raw_vocab.stoi, raw_vocab.itos
    def convert_tokens_to_ids(self, seq, vocab):
        out = [None]*len(seq)
        for i, tok in enumerate(seq):
            if tok.startswith('http://'):
                out[i] = 1
#             if tok.
            if tok in vocab:
                out[i] = vocab.index(tok)
            else:
                out[i] = 0
        return out

In [281]:
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [282]:
tokenizer = data.get_tokenizer('spacy')

UNK_TOKEN = "<unk>"
PAD_TOKEN = "<pad>"    
SOS_TOKEN = "<s>"
EOS_TOKEN = "</s>"
LOWER = True

# we include lengths to provide to the RNNs
SRC = data.Field(tokenize=tokenizer, 
                      lower=True, include_lengths=True, batch_first=True, 
                      unk_token=UNK_TOKEN, pad_token=PAD_TOKEN, init_token=None, eos_token=EOS_TOKEN)
TRG = data.Field(tokenize=tokenizer, 
                      lower=True, include_lengths=True, batch_first=True,
                      unk_token=UNK_TOKEN, pad_token=PAD_TOKEN, init_token=None, eos_token=EOS_TOKEN)


MAX_LEN = 500  # NOTE: we filter out a lot of sentences for speed
trn_data_fields = [("src", SRC), ("trg", TRG)]
train_data, valid_data, test_data = data.TabularDataset.splits(path=f'{DATA_PATH}',
                                 train='train.csv', validation='val.csv', test='test.csv',
                                 format='csv', skip_header=True, fields=trn_data_fields, 
                                 filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and len(vars(x)['trg']) <= MAX_LEN)



MIN_FREQ = 5  # NOTE: we limit the vocabulary to frequent words for speed
SRC.build_vocab(train_data.src, min_freq=MIN_FREQ)
TRG.build_vocab(train_data.trg, min_freq=MIN_FREQ)

PAD_INDEX = TRG.vocab.stoi[PAD_TOKEN]

In [295]:
train_data.fields['src'].vocab = glove_vocab_modify

In [297]:
train_data.fields['trg'].vocab = glove_vocab_modify

In [328]:
train_data.fields['trg'].vocab.vectors.shape

torch.Size([100002, 300])

In [330]:
from torch import nn

In [334]:
nn.Embedding(2,10)

Embedding(2, 10)

In [340]:
nn.Embedding.from_pretrained(glove_vocab_modify.vectors)

Embedding(100002, 300)

In [316]:
train_data[0]

In [318]:
DEVICE = 'cpu'

In [319]:
train_iter = data.BucketIterator(train_data, batch_size=20, train=True, 
                                 sort_within_batch=True, 
                                 sort_key=lambda x: (len(x.src), len(x.trg)), repeat=False,
                                 device=DEVICE)
valid_iter = data.Iterator(valid_data, batch_size=1, train=False, sort=False, repeat=False, 
                           device=DEVICE)